In [19]:
import pandas as pd
from swmm_api.input_file import read_inp_file
from swmm_api.input_file.section_labels import *
from swmm_api.input_file.sections import *
from swmm_api.input_file.helpers import *
from swmm_api.input_file.sections.others import TimeseriesData
from swmm_api.input_file.section_labels import TIMESERIES
import os
os.environ['CONDA_DLL_SEARCH_MODIFICATION_ENABLE']="1"
import numpy as np
import shutil
import yaml
from pyswmm import Simulation,Links,Nodes,RainGages,SystemStats
import matplotlib.pyplot as plt

config = yaml.load(open("./states_yaml/chaohu.yaml"), yaml.FullLoader)
import step_sim
import GI_setting as gs

### 获取建筑屋顶面积，设置GI，保存inp
def add_GI(inp):
    roof=pd.read_csv('polygon.csv')
    unit_area=10
    for i in range(roof.shape[0]):
        number=int(0.5*roof[['roofarea']].iloc[i][0]/unit_area)
        #print(roof[['Name']].iloc[i][0],number)
        # added line of LID_USAGE
        item=gs.LID_setting(roof[['Name']].iloc[i][0],'GreenRoof',number,10,0,0,0,0,'*','*',0)
        inp['LID_USAGE'][roof[['Name']].iloc[i][0]]=item

In [28]:
params={
    'kf':1e3,
    'kc':1e3,
}

# S1: RSH

In [29]:
raine = np.load('./RSH/east.npy').tolist()
rainw = np.load('./RSH/west.npy').tolist()
inp1 = read_inp_file('./S1/chaohu_.inp')

for rainid in range(6):
    inp1[TIMESERIES]['rainfalle']=TimeseriesData('rainfalle',raine[rainid])
    inp1[TIMESERIES]['rainfallw']=TimeseriesData('rainfallw',rainw[rainid])
    inp1.write_file('./S1/Results/chaohu_rain_'+str(rainid)+'.inp')
    step_sim.simfile('S1','./S1/Results/chaohu_rain_'+str(rainid)+'.inp',rainid,config,params)


# S2: R

In [30]:
rain = np.load('./RealRain/real.npy').tolist()
inp1 = read_inp_file('./S2/chaohu_.inp')

for rainid in range(len(rain)):    
    inp1[TIMESERIES]['rainfalle']=TimeseriesData('rainfalle',rain[rainid])
    inp1[TIMESERIES]['rainfallw']=TimeseriesData('rainfallw',rain[rainid])
    inp1.write_file('./S2/Results/chaohu_rain_'+str(rainid)+'.inp')
    step_sim.simfile('S2','./S2/Results/chaohu_rain_'+str(rainid)+'.inp',rainid,config,params)

# S3: RSH-GR

In [31]:
raine = np.load('./RSH/east.npy').tolist()
rainw = np.load('./RSH/east.npy').tolist()

for rainid in range(6):
    inp = read_inp_file('./S3/chaohu_GI.inp')
    # GI
    #add_GI(inp)
    # rain
    inp[TIMESERIES]['rainfalle']=TimeseriesData('rainfalle',raine[rainid])
    inp[TIMESERIES]['rainfallw']=TimeseriesData('rainfallw',rainw[rainid])
    inp.write_file('./S3/Results/chaohu_GI_rain_'+str(rainid)+'.inp')
    step_sim.simfile('S3','./S3/Results/chaohu_GI_rain_'+str(rainid)+'.inp',rainid,config,params)

# S4: R-GR

In [32]:
rain = np.load('./RealRain/real.npy').tolist()

for rainid in range(len(rain)):
    inp = read_inp_file('./S4/chaohu_GI.inp')
    # GI
    #add_GI(inp)
    # rain
    inp[TIMESERIES]['rainfalle']=TimeseriesData('rainfalle',rain[rainid])
    inp[TIMESERIES]['rainfallw']=TimeseriesData('rainfallw',rain[rainid])
    inp.write_file('./S4/Results/chaohu_GI_noRTC_rain_'+str(rainid)+'.inp')
    step_sim.simfile('S4','./S4/Results/chaohu_GI_noRTC_rain_'+str(rainid)+'.inp',rainid,config,params)
    

# S5: RSH-HC

In [33]:
raine = np.load('./RSH/east.npy').tolist()
rainw = np.load('./RSH/east.npy').tolist()
inp1 = read_inp_file('./S5/chaohu_RTC.inp')

for rainid in range(6):
    inp1[TIMESERIES]['rainfalle']=TimeseriesData('rainfalle',raine[rainid])
    inp1[TIMESERIES]['rainfallw']=TimeseriesData('rainfallw',rainw[rainid])
    inp1.write_file('./S5/Results/chaohu_rain_'+str(rainid)+'.inp')
    step_sim.simfile('S5','./S5/Results/chaohu_rain_'+str(rainid)+'.inp',rainid,config,params)


# S6: R-HC

In [34]:
rain = np.load('./RealRain/real.npy').tolist()
inp1 = read_inp_file('./S6/chaohu_RTC.inp')

for rainid in range(len(rain)):    
    inp1[TIMESERIES]['rainfalle']=TimeseriesData('rainfalle',rain[rainid])
    inp1[TIMESERIES]['rainfallw']=TimeseriesData('rainfallw',rain[rainid])
    inp1.write_file('./S6/Results/chaohu_rain_'+str(rainid)+'.inp')
    step_sim.simfile('S6','./S6/Results/chaohu_rain_'+str(rainid)+'.inp',rainid,config,params)

# S7: RSH-GR-HC

In [35]:
raine = np.load('./RSH/east.npy').tolist()
rainw = np.load('./RSH/east.npy').tolist()

for rainid in range(6):
    inp = read_inp_file('./S7/chaohu_GI_RTC.inp')
    # GI
    #add_GI(inp)
    # rain
    inp[TIMESERIES]['rainfalle']=TimeseriesData('rainfalle',raine[rainid])
    inp[TIMESERIES]['rainfallw']=TimeseriesData('rainfallw',rainw[rainid])
    inp.write_file('./S7/Results/chaohu_GI_rain_'+str(rainid)+'.inp')
    step_sim.simfile('S7','./S7/Results/chaohu_GI_rain_'+str(rainid)+'.inp',rainid,config,params)

# S8: R-GR-HC

In [ ]:
rain = np.load('./RealRain/real.npy').tolist()

for rainid in range(len(rain)):
    inp = read_inp_file('./S8/chaohu_GI_RTC.inp')
    # GI
    #add_GI(inp)
    # rain
    inp[TIMESERIES]['rainfalle']=TimeseriesData('rainfalle',rain[rainid])
    inp[TIMESERIES]['rainfallw']=TimeseriesData('rainfallw',rain[rainid])
    inp.write_file('./S8/Results/chaohu_GI_rain_'+str(rainid)+'.inp')
    step_sim.simfile('S8','./S8/Results/chaohu_GI_rain_'+str(rainid)+'.inp',rainid,config,params)
    